In [1]:
!bash -c 'source activate tl-detect && conda install -qy -c conda-forge keras'

Solving environment: ...working... done


==> WARNING: A newer version of conda exists. <==
  current version: 4.4.10
  latest version: 4.5.0

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /opt/conda/envs/tl-detect

  added / updated specs: 
    - keras


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    h5py-2.7.1                 |           py35_2         3.9 MB  conda-forge
    keras-2.1.5                |           py35_0         498 KB  conda-forge
    yaml-0.1.7                 |                0         302 KB  conda-forge
    pygpu-0.7.5                |           py35_0         1.4 MB  conda-forge
    mako-1.0.7                 |           py35_0         115 KB  conda-forge
    scipy-1.0.1                |py35_blas_openblas_200        39.4 MB  conda-forge
    markupsafe-1.0             |           py35_0          32

In [29]:
!cat ~/.keras/keras.json

{
    "backend": "tensorflow",
    "image_data_format": "channels_last",
    "epsilon": 1e-07,
    "floatx": "float32"
}

In [24]:
!cp -R ../imgs/training/bosch/Green ../imgs/training/ && cp -R ../imgs/training/bosch/Red ../imgs/training/ && cp -R ../imgs/training/bosch/Yellow ../imgs/training/  && mv ../imgs/training/bosch ../imgs/bosch_tl 

cp: cannot stat '../imgs/training/bosch/Green': No such file or directory


In [ ]:
! cd ../imgs/training && find . -size 0 -exec rm {} \;

In [34]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = '../imgs/training/'
validation_data_dir = '../imgs/validation'
nb_train_samples = 10000
nb_validation_samples = 500
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


150

In [42]:

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=20)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None)

validation_generator = ImageDataGenerator().flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None)



Found 7246 images belonging to 3 classes.
Found 893 images belonging to 3 classes.


In [43]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('first_try.h5')

Epoch 1/50


ValueError: Output of generator should be a tuple `(x, y, sample_weight)` or `(x, y)`. Found: [[[[129.       162.       167.      ]
   [129.       162.       167.      ]
   [129.       162.       167.      ]
   ...
   [123.       159.       171.      ]
   [123.       159.       171.      ]
   [123.       159.       171.      ]]

  [[128.3724   161.74896  168.12967 ]
   [129.       162.       167.      ]
   [129.       162.       167.      ]
   ...
   [123.       159.       171.      ]
   [123.       159.       171.      ]
   [123.       159.       171.      ]]

  [[124.       160.       176.      ]
   [124.23935  160.09573  175.56917 ]
   [125.27956  160.51183  173.6968  ]
   ...
   [123.       159.       171.      ]
   [123.       159.       171.      ]
   [123.       159.       171.      ]]

  ...

  [[128.       168.       159.      ]
   [128.       168.       159.      ]
   [128.       168.       159.      ]
   ...
   [117.39731  149.60269  131.      ]
   [117.605354 149.39465  131

In [15]:
nb_validation_samples // batch_size

55